In [1]:
import numpy as np
import pickle
# from Utils.TimeLogger import log
from scipy.sparse import csr_matrix
import time
import scipy.sparse as sp
minn = 1647180684
maxx = 0
import json
year_dict=dict()
def ok(year,month):
    global year_dict
    year_dict[year]=1
    if year == 2014 :
        return True
    else:
        return False
def transTime(date):
    timeArr = time.localtime(date)
    # timeArr = time.strptime(timeArray, '%Y-%m-%d %H:%M:%S')
    year = date# = time.mktime(timeArr)
    global minn
    global maxx
    # global minn_y
    # global maxx_y
    # minn_y = min(minn_y,timeArr.tm_year)
    # maxx_y = max(maxx_y,timeArr.tm_year)
    if (ok(timeArr.tm_year,timeArr.tm_mon)):
        minn = min(minn, year)
        maxx = max(maxx, year)        
        return year
    return None# time.mktime(timeArr)

In [2]:
def mapping(infile):
	global minn
	global maxx
	usrId = dict()
	itmId = dict()
	usrid, itmid = [0, 0]
	interaction = list()
	with open(infile, 'r') as fs:
		for line in fs:
			arr = line.strip().split(',')
			row = arr[0]
			col = arr[1]
			timeStamp = transTime(int(arr[-1]))
			if(timeStamp==None):
				continue
			if row not in usrId:
				usrId[row] = usrid
				interaction.append(dict())
				usrid += 1
			if col not in itmId:
				itmId[col] = itmid
				itmid += 1
			usr = usrId[row]
			itm = itmId[col]
			if(itm not in interaction[usr]):
				interaction[usr][itm]=[]
			interaction[usr][itm].extend([timeStamp])
	print(minn, maxx)
	return interaction, usrid, itmid, itmId

In [7]:
def checkFunc1(cnt):
	return cnt >= 20
def checkFunc2(cnt):
	return cnt >= 15
def checkFunc3(cnt):
	return cnt >= 5
def checkFunc4(cnt):
    return cnt <= 200
def filter(interaction, usrnum, itmnum, ucheckFunc, icheckFunc, filterItem=True):
    # get keep set
    usrKeep = set()
    itmKeep = set()
    itmCnt = [0] * itmnum
    for usr in range(usrnum):
        data = interaction[usr]
        usrCnt = 0
        for col in data:
            itmCnt[col] += 1
            usrCnt += 1
        if ucheckFunc(usrCnt):
            usrKeep.add(usr)
    for itm in range(itmnum):
        if not filterItem or icheckFunc(itmCnt[itm]):
            itmKeep.add(itm)

    # filter data
    retint = list()
    usrid = 0
    itmid = 0
    itmId = dict()
    user_id = list()
    item_id = list()
    time_int = list()
    for row in range(usrnum):
        if row not in usrKeep:
            continue
        usr = usrid
        usrid += 1
        retint.append(dict())
        data = interaction[row]
        for col in data:
            if col not in itmKeep:
                continue
            if col not in itmId:
                itmId[col] = itmid
                itmid += 1
            itm = itmId[col]
            retint[usr][itm] = data[col]
            for time_div in data[col]:
                user_id.append(usr+1)
                item_id.append(itm+1)
                time_int.append(int(time_div))
    return retint, usrid, itmid, user_id, item_id, time_int, itmId

In [3]:
def negSamp(temLabel, sampSize, nodeNum):
    negset = [None] * sampSize
    cur = 0
    i = 0
    while cur < sampSize:
        # rdmItm = neg_frequency[-i]# 
        rdmItm=np.random.choice(nodeNum)
        if rdmItm not in temLabel:
            negset[cur] = rdmItm+1
            cur += 1
        i += 1
    return negset

def split_test(interaction, usrnum, itmnum):
    np.random.seed(100)
    pickNum = 10000
    # random pick
    usrPerm = np.random.permutation(usrnum)
    pickUsr = usrPerm[:pickNum]

    tstInt = [None] * usrnum
    exception = 0
    user_id=list()
    item_id=list()
    time_id=list()
    neg_items=list()
    for usr in pickUsr:
        temp = list()
        data = interaction[usr]
        for itm in data:
            temp.append((itm, data[itm]))
        if len(temp) == 0:
            exception += 1
            continue
        temp.sort(key=lambda x: x[1])
        tstInt[usr] = temp[-1][0]
        user_id.append(usr+1)
        item_id.append(tstInt[usr]+1)
        time_id.append(int(interaction[usr][tstInt[usr]][-1]))
        neg_items.append(negSamp(interaction[usr],999,itmnum))
        # interaction[usr][tstInt[usr]] = None
    # print(user_id,time_id,time_id,neg_items)
    train=DataFrame({'user_id':user_id, 'item_id':item_id, 'time':time_id, 'neg_items':neg_items})
    train.to_csv('./Datasets/amazon/test_amazon.csv',index=False,sep='\t')
    # print(train)
    print('Exception:', exception, np.sum(np.array(tstInt)!=None))
    return interaction, tstInt

In [4]:
def split(interaction, usrnum, itmnum):
    np.random.seed(100)
    pickNum = 10000
    # random pick
    usrPerm = np.random.permutation(usrnum)
    pickUsr = usrPerm[:pickNum]
    tstInt = [None] * usrnum
    trainPos = [None]* usrnum
    exception = 0
    user_id=list()
    item_id=list()
    time_id=list()
    neg_items=list()
    np.random.seed(100)
    for usr in pickUsr:
        temp = list()
        data = interaction[usr]
        for itm in data:
            if (itm is not None and data[itm] is not None):
                temp.append((itm, data[itm]))
        if len(temp) == 0:
            print(usr)
            exception += 1
            continue
        # print('1',temp)
        # print(temp)
        temp.sort(key=lambda x: x[1])
        # print(temp)
        tstInt[usr] = temp[-1][0]
        # user_id.append(usr+1)
        # item_id.append(tstInt[usr]+1)
        # time_id.append(int(interaction[usr][tstInt[usr]][-1]))
        # neg_items.append(negSamp(interaction[usr],99,itmnum))
        # interaction[usr][tstInt[usr]] = None
        # print(usr,tstInt[usr],interaction[usr][tstInt[usr]])
        if(len(interaction[usr][tstInt[usr]])==1):
            interaction[usr][tstInt[usr]] = None
        else:
            interaction[usr][tstInt[usr]] = interaction[usr][tstInt[usr]][1:]
    user_behavior_sequence=list()
    for usr in range(usrnum):
        temp = list()
        data = interaction[usr]
        for itm in data:
            if(itm is not None and data[itm] is not None):
                temp.append((itm, data[itm]))
        if len(temp) == 0:
            print('2',usr)
            exception += 1
            continue
        #print(temp)
        temp.sort(key=lambda x: x[1])
        user_behavior_sequence.append(np.array(temp)[:-1,0])
        trainPos[usr] = temp[-1][0]
        # interaction[usr][trainPos[usr]] = None
        # print(usr,trainPos[usr],interaction[usr][trainPos[usr]])
        if(len(interaction[usr][trainPos[usr]])==1):
            interaction[usr][trainPos[usr]] = None
        else:
            interaction[usr][trainPos[usr]] = interaction[usr][trainPos[usr]][1:]
    # print(user_id,time_id,time_id,neg_items)
    # train=DataFrame({'user_id':user_id, 'item_id':item_id, 'time':time_id, 'neg_items':neg_items})
    # train.to_csv('test_yelp_4.csv',index=False,sep='\t')
    # print(train)
    print('Exception:', exception, np.sum(np.array(tstInt)!=None))
    return interaction, tstInt, trainPos, user_behavior_sequence

In [5]:
from pandas.core.frame import DataFrame
prefix = 'D:/edgedownload/'
print('Start')
minn = 1647180684
maxx = 0
minn_y = 2022
maxx_y = 2000
interaction, usrnum, itmnum,itemID = mapping(prefix + 'amazon_ratings_Books.csv')
print('Id Mapped, usr %d, itm %d' % (usrnum, itmnum))

Start
1388534400 1406073600
Id Mapped, usr 2234577, itm 831117


In [8]:
checkFuncs = [checkFunc1, checkFunc2, checkFunc3]
itmID_filter=[itemID]
for i in range(len(checkFuncs)):
    filterItem = True if i < 1 else False
    interaction, usrnum, itmnum, user_id, item_id, time_int,itmID_filter_one = filter(interaction, usrnum, itmnum, checkFuncs[i], checkFuncs[i], filterItem)
    print('Filter', i, 'times:', usrnum, itmnum)
    itmID_filter.append(itmID_filter_one)
    '''
    if(i==len(checkFuncs)):
        train=DataFrame({'user_id':user_id, 'item_id':item_id, 'time':time_int})
        train.to_csv('train_amazon.csv',index=False,sep='\t')
        print(train)
    '''
        
print('Sparse Samples Filtered, usr %d, itm %d' % (usrnum, itmnum))
print(len(interaction))


Filter 0 times: 17064 33325
Filter 1 times: 11199 30821
Filter 2 times: 11199 30821
Sparse Samples Filtered, usr 11199, itm 30821
11199


In [9]:
def trans_sub(interaction, usrnum, itmnum, gragh_num):
	global minn
	global maxx
	interval = (maxx-minn)/gragh_num
	rcd = [[list(), list(), list()]]
	for i in range(gragh_num-1):
		rcd.append([list(), list(), list()])
	timeMat = sp.dok_matrix((usrnum, itmnum), dtype=np.int)
	for usr in range(usrnum):
		if interaction[usr] == None:
			continue
		data = interaction[usr]
		for col in data:
			if data[col] != None:
				for one_data in data[col]:
					gragh_no=int(((one_data-minn)/interval))
					if(gragh_no>=gragh_num):
						print(one_data,gragh_no)
						gragh_no=gragh_num-1
					# print(gragh_no,one_data)
					if(len(rcd[gragh_no][0])==0 or  rcd[gragh_no][0][-1]!=usr or rcd[gragh_no][1][-1]!=col):
						rcd[gragh_no][0].append(usr)
						rcd[gragh_no][1].append(col)
						rcd[gragh_no][2].append(one_data)
						# rcd[gragh_no][0].append(usrnum+col)
						# rcd[gragh_no][1].append(usr)
						# rcd[gragh_no][2].append(1.0)
						timeMat[usr,col]=gragh_no
	intMat=list()
	for i in range(gragh_num):
		intMat.append(csr_matrix((rcd[i][2], (rcd[i][0], rcd[i][1])), shape=(usrnum, itmnum)))
		# intMat.append(normalized_adj_single(csr_matrix((rcd[i][2], (rcd[i][0], rcd[i][1])), shape=(usrnum+itmnum, usrnum+itmnum))))+itmnum usrnum+
		print(intMat[i])
	return intMat, timeMat.tocsr()

In [10]:
interaction_copy = interaction.copy()
interaction_copy, tstInt=split_test(interaction_copy, usrnum, itmnum)
def transtocsv(interaction, usrnum, itmnum):
    user_id = list()
    item_id = list()
    time_int = list()
    for row in range(usrnum):
        data = interaction[row]
        # print(data)
        for col in data:
            # print(col)
            if(data[col] is None):
                continue
            for time_div in data[col]:
                user_id.append(row+1)
                item_id.append(col+1)
                time_int.append(int(time_div))
    return user_id,item_id,time_int
user_id, item_id, time_int = transtocsv(interaction_copy, usrnum, itmnum)
train=DataFrame({'user_id':user_id, 'item_id':item_id, 'time':time_int})
train.to_csv('./Datasets/amazon/train_amazon.csv',index=False,sep='\t')

Exception: 0 10000


In [11]:
# print(interaction, usrnum, itmnum), trainPos
trnInt, tstInt, trainPos, user_behavior_sequence = split(interaction, usrnum, itmnum)
print('Datasets Splited')
# print(trnInt)

C:\Users\LIU-YUXI\AppData\Local\Temp/ipykernel_29520/939746082.py:53: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  user_behavior_sequence.append(np.array(temp)[:-1,0])


Exception: 0 10000
Datasets Splited


In [12]:
print(len(trnInt))

11199


In [13]:
tstInt

[21,
 44,
 None,
 70,
 148,
 166,
 None,
 None,
 223,
 243,
 284,
 379,
 441,
 240,
 479,
 499,
 291,
 617,
 None,
 639,
 666,
 698,
 785,
 819,
 None,
 11,
 862,
 882,
 914,
 919,
 1002,
 1020,
 None,
 None,
 1093,
 1108,
 1136,
 None,
 None,
 1256,
 1270,
 1299,
 1230,
 1331,
 1352,
 1377,
 1450,
 1460,
 1492,
 1549,
 1569,
 1596,
 1602,
 114,
 1655,
 1722,
 1766,
 1787,
 1799,
 1827,
 None,
 1375,
 None,
 1890,
 1910,
 1921,
 1939,
 1971,
 553,
 2022,
 2052,
 2068,
 2087,
 2106,
 2189,
 2230,
 1318,
 2318,
 494,
 None,
 240,
 656,
 2413,
 None,
 2443,
 403,
 2483,
 None,
 2559,
 2752,
 2767,
 2772,
 2790,
 2832,
 2850,
 2939,
 3001,
 754,
 3061,
 None,
 None,
 2466,
 261,
 3154,
 3171,
 3184,
 3191,
 3248,
 3267,
 None,
 3343,
 3350,
 None,
 3433,
 3451,
 3471,
 89,
 1117,
 3507,
 884,
 None,
 3560,
 3581,
 3534,
 3618,
 3629,
 992,
 3671,
 3686,
 3724,
 3737,
 103,
 None,
 3779,
 3799,
 3771,
 None,
 111,
 3891,
 3926,
 3979,
 4026,
 1210,
 4091,
 4122,
 4147,
 4153,
 4171,
 None,


In [14]:
with open('./Datasets/amazon/tst_int', 'wb') as fs:
	pickle.dump(tstInt, fs)

In [15]:
minn = 1647180684
maxx = 0

def trans(interaction, usrnum, itmnum):
	global maxx,minn
	count=0
	r, c, d = [list(), list(), list()]
	for usr in range(usrnum):
		if interaction[usr] == None:
			count+=1
			continue
		data = interaction[usr]
		for col in data:
			if data[col] != None:
				for one_data in data[col]:
					if(one_data>maxx):
						maxx=one_data
					if(one_data<minn):
						minn=one_data
					r.append(usr)
					c.append(col)
					d.append(1.0)
				else:
					count+=1
	print(max(r),max(c))
	print(count)
	intMat = csr_matrix((d, (r, c)), shape=(usrnum, itmnum))
	return intMat

In [16]:
print(usrnum,itmnum)
trnMat = [trans(trnInt, usrnum, itmnum)]
print(maxx,minn)
subMat,timeMat=trans_sub(trnInt, usrnum, itmnum, 5)
print(timeMat)
trnMat.append(subMat)
trnMat.append(timeMat)
print('Train Mat Done')
with open('./Datasets/amazon/trn_mat_time', 'wb') as fs:
	pickle.dump(trnMat, fs)
fs.close()

11199 30821
11198 30820
354716
1406073600 1388534400


C:\Users\LIU-YUXI\AppData\Local\Temp/ipykernel_29520/2970320320.py:8: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  timeMat = sp.dok_matrix((usrnum, itmnum), dtype=np.int)


1406073600 5
1406073600 5
1406073600 5
  (0, 17)	1391731200
  (0, 18)	1388534400
  (0, 30)	1389484800
  (1, 0)	1389830400
  (1, 33)	1389225600
  (1, 36)	1389830400
  (1, 38)	1389830400
  (1, 43)	1389225600
  (2, 49)	1390608000
  (2, 54)	1391385600
  (2, 56)	1391817600
  (6, 67)	1389830400
  (6, 189)	1389657600
  (6, 192)	1391126400
  (6, 199)	1390176000
  (6, 200)	1391817600
  (7, 198)	1389225600
  (8, 67)	1389657600
  (8, 214)	1388620800
  (8, 217)	1389657600
  (8, 220)	1389657600
  (8, 222)	1389657600
  (8, 225)	1389657600
  (8, 229)	1389657600
  (9, 67)	1389398400
  :	:
  (11193, 29211)	1390435200
  (11193, 30598)	1390348800
  (11193, 30680)	1390348800
  (11194, 13820)	1391731200
  (11194, 14046)	1389225600
  (11194, 14199)	1390867200
  (11194, 16563)	1391558400
  (11194, 22905)	1389657600
  (11194, 26806)	1390694400
  (11194, 27610)	1391990400
  (11194, 29211)	1390953600
  (11194, 29212)	1391472000
  (11194, 30598)	1390780800
  (11194, 30678)	1390176000
  (11194, 30679)	1389830400


In [17]:
trnMat

[<11199x30821 sparse matrix of type '<class 'numpy.float64'>'
 	with 354716 stored elements in Compressed Sparse Row format>,
 [<11199x30821 sparse matrix of type '<class 'numpy.intc'>'
  	with 72280 stored elements in Compressed Sparse Row format>,
  <11199x30821 sparse matrix of type '<class 'numpy.intc'>'
  	with 78997 stored elements in Compressed Sparse Row format>,
  <11199x30821 sparse matrix of type '<class 'numpy.intc'>'
  	with 79692 stored elements in Compressed Sparse Row format>,
  <11199x30821 sparse matrix of type '<class 'numpy.intc'>'
  	with 78096 stored elements in Compressed Sparse Row format>,
  <11199x30821 sparse matrix of type '<class 'numpy.intc'>'
  	with 45651 stored elements in Compressed Sparse Row format>],
 <11199x30821 sparse matrix of type '<class 'numpy.intc'>'
 	with 282436 stored elements in Compressed Sparse Row format>]